# Feature Selection

## Imports and preprocessing

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import arviz as az
import pymc as pm
from zipfile import ZipFile
from sklearn.model_selection import train_test_split
from sklearn.metrics import RocCurveDisplay, auc, roc_curve

WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:
RANDOM_SEED = 8927
rng = np.random.default_rng(RANDOM_SEED)

In [3]:
# read in data
zip_file = ZipFile("data/adult23csv.zip")

df_full = pd.read_csv(zip_file.open('adult23.csv'))
y_name = 'LONGCOVD1_A'


In [4]:
from preprocessing import preprocess_all_columns

X_train, X_test, y_train, y_test = preprocess_all_columns()

In [5]:
## HOW TO GET 9.0 and 7.0 COLUMN NAMES:
X_train.filter(regex='_9\.0$').columns
X_train.filter(regex='_7\.0$').columns

Index(['RATCAT_A_7.0', 'HISPALLP_A_7.0', 'RACEALLP_A_7.0', 'CITZNSTP_A_7.0',
       'SPOUSEDUCP_A_7.0', 'MARSTAT_A_7.0', 'SPOUSWRK_A_7.0',
       'PCNTADWFP1_A_7.0', 'PCNTADWKP1_A_7.0', 'EMPWRKFT1_A_7.0',
       ...
       'STREV_A_7.0', 'MIEV_A_7.0', 'ANGEV_A_7.0', 'CHDEV_A_7.0',
       'CHLEV_A_7.0', 'HYPEV_A_7.0', 'LSATIS4_A_7.0', 'PHSTAT_A_7.0',
       'AVAIL_A_7.0', 'INTV_MON_7.0'],
      dtype='object', length=238)

In [6]:
N, D = X_train.shape

coords = {"predictors" : X_train.columns.values}

# # IT IS INTRACTIBLE TO RUN IT THIS WAY
# with pm.Model(coords=coords) as sparse_model:
#     X = X_train.values
#     lamda = pm.HalfCauchy('lambda', beta=1, dims="predictors")
#     tau = pm.HalfCauchy('tau', beta=1)
#     sigma = pm.Deterministic('horseshoe', tau*tau*lamda*lamda)
#     beta = pm.Normal('beta', mu=0, sigma=sigma, dims="predictors")
#     beta0 = pm.Normal('beta0', mu=0, sigma=10)
#     mu = pm.math.invlogit(pm.math.dot(X, beta) + beta0)
#     y = pm.Bernoulli('obs', p=mu, observed=y_train.values)
# sparse_model


## SPARSITY ENFORCING PRIOR: https://www.pymc.io/projects/docs/en/stable/learn/core_notebooks/pymc_overview.html
import pytensor.tensor as pt

N, D = X_train.shape
D0 = 10 # guess at the true number of predictors

with pm.Model(coords={"predictors": X_train.columns.values}) as sparse_model:

    x = X_train.values
    
    # Prior on error SD
    sigma = pm.HalfNormal("sigma", 10)

    # Global shrinkage prior
    tau = pm.HalfStudentT("tau", 2, D0 / (D - D0) * sigma / np.sqrt(N))
    # Local shrinkage prior
    lam = pm.HalfStudentT("lam", 4, dims="predictors")
    c2 = pm.InverseGamma("c2", 1, 1)
    z = pm.Normal("z", 0.0, 1.0, dims="predictors")
    # Shrunken coefficients
    betas = pm.Deterministic(
        "beta", z * tau * lam * pt.sqrt(c2 / (c2 + tau**2 * lam**2)), dims="predictors"
    )
    # No shrinkage on intercept
    b0 = pm.Normal("beta0", 0, 100)

    # Model
    logitp = b0 + pm.math.dot(x, betas)
    theta = 1 / (1 + pm.math.exp(-logitp))
    y = pm.Bernoulli('y', p=theta, observed=y_train.values)

sparse_model

sigma ~ HalfNormal(0, 10)
  tau ~ HalfStudentT(2, f(sigma))
  lam ~ HalfStudentT(4, 1)
   c2 ~ InverseGamma(1, 1)
    z ~ Normal(0, 1)
beta0 ~ Normal(0, 100)
 beta ~ Deterministic(f(lam, z, c2, tau))
    y ~ Bernoulli(f(beta0, lam, z, c2, tau))

In [7]:
# run MCMC to get posterior distribution
with sparse_model:
    idata_sparse = pm.sample(1000, tune=1500, target_accept=0.9)
    print(f'DONE')

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [sigma, tau, lam, c2, z, beta0]


Output()

ValueError: Not enough samples to build a trace.

In [ ]:
summary = az.summary(idata_sparse, var_names=["betas"], hdi_prob=0.95)
summary

In [ ]:
az.plot_forest(idata_sparse, var_names=["betas"], combined=True, hdi_prob=0.95, r_hat=False, rope=(0,0), textsize=8);
plt.tight_layout()

In [ ]:
cutoff = 0.001
np.abs(summary['mean']).sort_values(ascending=False)[:20].index.values

In [ ]:
import re
# extract top 20 features but drop duplicates
important = np.abs(summary['mean']).sort_values(ascending=False)[:20].index.values
important_feats = [re.search('\[(.+)_\d+\.0\]',idx).group(1) for idx in important]
# # preserve order
important_feats = list(dict.fromkeys(important_feats)) 
print(important_feats)

pattern = '|'.join(important_feats)
# # filter to keep only relevant features
X_train_new = X_train.filter(regex=pattern)
X_test_new = X_test.filter(regex=pattern)